In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_line_cnt = 0
test_line_cnt = 0
valid_line_cnt = 0
train_line_lst = []
test_line_lst = []
val_line_lst = []

#SET.sent contains the sentences, one sentence per line.
with open("/content/drive/Shared drives/DSCI531/wikipedia-biography-dataset/train/train.sent") as f_tr:
    for line in f_tr:
        train_line_cnt += 1
        train_line_lst += [line]
print(train_line_cnt)

with open("/content/drive/Shared drives/DSCI531/wikipedia-biography-dataset/test/test.sent") as f_t:
    for line in f_t:
        test_line_cnt += 1
        test_line_lst += [line]
print(test_line_cnt)

with open("/content/drive/Shared drives/DSCI531/wikipedia-biography-dataset/valid/valid.sent") as f_v:
    for line in f_v:
        valid_line_cnt += 1
        val_line_lst += [line]
print(valid_line_cnt)

2395103
298900
299273


In [ ]:
train_line_lst[:5]

['walter extra is a german award-winning aerobatic pilot , chief aircraft designer and founder of extra flugzeugbau -lrb- extra aircraft construction -rrb- , a manufacturer of aerobatic aircraft .\n',
 'extra was trained as a mechanical engineer .\n',
 'he began his flight training in gliders , transitioning to powered aircraft to perform aerobatics .\n',
 'he built and flew a pitts special aircraft and later built his own extra ea-230 .\n',
 'extra began designing aircraft after competing in the 1982 world aerobatic championships .\n']

In [ ]:
len(train_line_lst)

2395103

In [ ]:
gender_lst = ['she', 'her', 'hers', 'mommy', 'mom', 'girl', 'mother', 'lady', 'sister', 'mama', 'momma', 'sis', 'grandma', 'herself', \
              'woman', 'women', 'female', 'he', 'his', 'him', 'daddy', 'dad', 'boy', 'father', 'guy', 'brother', 'dada', 'papa', 'bro',
              'grandpa', 'himself', 'man', 'men', 'male']
female_lst = ['she', 'her', 'hers', 'mommy', 'mom', 'girl', 'mother', 'lady', 'sister', 'mama', 'momma', 'sis', 'grandma', 'herself', 'woman', \
              'women', 'female']
male_lst = ['he', 'his', 'him', 'daddy', 'dad', 'boy', 'father', 'guy', 'brother', 'dada', 'papa', 'bro', 'grandpa', 'himself', \
            'man', 'men', 'male']

In [ ]:
train_line_lst = [x.lower() for x in train_line_lst]
test_line_lst = [x.lower() for x in test_line_lst]
val_line_lst = [x.lower() for x in val_line_lst]

In [ ]:
# First filter out the sentences with gender words for faster runtime later
filtered_train = [x for x in train_line_lst if any(word in x.split() for word in gender_lst)]
filtered_test = [x for x in test_line_lst if any(word in x.split() for word in gender_lst)]
filtered_val = [x for x in val_line_lst if any(word in x.split() for word in gender_lst)]

In [ ]:
print(len(filtered_train))
print(len(filtered_test))
print(len(filtered_val))

1349103
167707
168769


In [ ]:
#pip install nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Word Stimuli Table

In [ ]:
good_lst = ['happiness', 'happy', 'fun', 'fantastic', 'lovable', 'magical', 'delight', 'joy', 'relaxing', 'honest', 'excited', \
            'laughter', 'cheerful', 'great', 'moral', 'kindly', 'upright', 'worthy', 'outstanding', 'admirable', 'gracious',\
            'valuable', 'excellent', 'exceptional', 'superb', 'wonderful']
bad_lst = ['torture', 'murder', 'abuse', 'wreck', 'die', 'disaster', 'mourning', 'killer', 'nightmare', \
           'stress', 'kill', 'death', 'deficient', 'horrible', 'wrong', 'unsatisfactory', 'dreadful', 'flawed', 'useless', \
           'worthless', 'immoral', 'nefarious', 'sinful', 'unethical', 'harmful', 'hostile']

In [ ]:
home_lst = ['baby', 'house', 'home', 'homes', 'wedding', 'kid', 'family', 'marry', 'domestic', 'household', 'households', \
            'chore', 'chores', 'families', 'kitchen', 'infant', 'babysit', 'childcare', 'parenting', 'garden', 'laundry',\
            'clean', 'cook']
work_lst = ['work', 'works', 'worked', 'labor', 'worker', 'workers', 'economy', 'office', 'job', 'jobs', 'business', 'businesses', \
            'trade', 'trades', 'company', 'companies', 'industry', 'industries', 'pay', 'pays', 'paid', 'working', 'salary', \
            'salaries', 'wage', 'wages', 'activity', 'act', 'money']

In [ ]:
art_lst = ['art', 'artist', 'artwork', 'dance', 'dancing', 'dancer', 'sing', 'singing', 'singer', 'paint', 'painting', 'painter', 'song', 'draw', 'drawing', 'craft', 'handcraft', 'handicraft', 'music', 'sculpture', 'design', 'photography']
science_lst = ['science', 'scientist', 'chemistry', 'physics', 'engineer', 'space', 'spaceship', 'astronaut', 'chemical', 'microscope', 'theory', 'physics', 'medicine', 'engineering', 'biology', 'technology', 'astrophysics', 'biochemistry', 'geology', 'laboratory', 'radiology']
literature_lst = ['book','books', 'novel', 'read', 'write', 'story', 'word', 'writing', 'reading', 'tale', 'history', 'poetry', 'poem', 'poet', 'literacy', 'written', 'prose', 'fiction', 'essay', 'author']
math_lst = ['puzzle', 'number', 'count', 'math', 'counting', 'calculator', 'subtraction', 'addition', "arithmetic", "calculation", "calculus", "ciphering", "computation", "mathematics", "geometry", "algebra"]

In [ ]:
from nltk.corpus.reader.wordlist import NonbreakingPrefixesCorpusReader
# Label sentences with female and male
def filer_gender(text):
    # If the sentence contains both gender pronouns, ignore the sentence
    if any(word in text.split() for word in female_lst) and any(word in text.split() for word in male_lst):
        return 'None'
    elif any(word in text.split() for word in female_lst):
        return 'Female'
    elif any(word in text.split() for word in male_lst):
        return 'Male'
    else:
        return 'None'

# Label sentences with "good" and "bad"
def filter_func1(text):
    # If the sentence contains words from both categories, ignore the sentence
    if any(word in text.split() for word in good_lst) and any(word in text.split() for word in bad_lst):
        return 'None'
    elif any(word in text.split() for word in good_lst):
        return 'Good'
    elif any(word in text.split() for word in bad_lst):
        return 'Bad'
    else:
        return 'None'
  
# Label the sentences with home, work, and other specific categories
def filter_func2(text):
    # Give only one label to each sentence
    if any(word in text.split() for word in home_lst):
        return 'home'
    elif any(word in text.split() for word in work_lst):
        return 'work'
    elif any(word in text.split() for word in art_lst):
        return 'art'
    elif any(word in text.split() for word in science_lst):
        return 'science'
    elif any(word in text.split() for word in literature_lst):
        return 'literature'
    elif any(word in text.split() for word in math_lst):
        return 'math'
    else:
        return 'None'

In [ ]:
# Create labeled dataframes
tmp1 = pd.DataFrame(filtered_train, columns = ['text'])
tmp1['gender'] = tmp1['text'].apply(filer_gender)
tmp1['category'] = tmp1['text'].apply(filter_func1)
labeled_train1 = tmp1[(tmp1['gender'] != 'None') & (tmp1['category'] != 'None')]
tmp2 = pd.DataFrame(filtered_test, columns = ['text'])
tmp2['gender'] = tmp2['text'].apply(filer_gender)
tmp2['category'] = tmp2['text'].apply(filter_func1)
labeled_test1 = tmp2[(tmp2['gender'] != 'None') & (tmp2['category'] != 'None')]
tmp3 = pd.DataFrame(filtered_val, columns = ['text'])
tmp3['gender'] = tmp3['text'].apply(filer_gender)
tmp3['category'] = tmp3['text'].apply(filter_func1)
labeled_val1 = tmp3[(tmp3['gender'] != 'None') & (tmp3['category'] != 'None')]
print(len(labeled_train1))
print(len(labeled_test1))
print(len(labeled_val1))

37009
4606
4657


In [ ]:
labeled_train1.head(5)

,text,gender,category
4,she was a member of the district committee of ...,Female,Bad
23,"on 5 november 1994 , he made english football ...",Male,Bad
57,he was a national selector at the time of his ...,Male,Bad
76,he believed his reputation as a hired killer w...,Male,Bad
116,"he advocates immigration restrictions , the de...",Male,Bad


In [ ]:
tmp1 = pd.DataFrame(filtered_train, columns = ['text'])
tmp1['gender'] = tmp1['text'].apply(filer_gender)
tmp1['category'] = tmp1['text'].apply(filter_func2)
labeled_train2 = tmp1[(tmp1['gender'] != 'None') & (tmp1['category'] != 'None')]
tmp2 = pd.DataFrame(filtered_test, columns = ['text'])
tmp2['gender'] = tmp2['text'].apply(filer_gender)
tmp2['category'] = tmp2['text'].apply(filter_func2)
labeled_test2 = tmp2[(tmp2['gender'] != 'None') & (tmp2['category'] != 'None')]
tmp3 = pd.DataFrame(filtered_val, columns = ['text'])
tmp3['gender'] = tmp3['text'].apply(filer_gender)
tmp3['category'] = tmp3['text'].apply(filter_func2)
labeled_val2 = tmp3[(tmp3['gender'] != 'None') & (tmp3['category'] != 'None')]
print(len(labeled_train2))
print(len(labeled_test2))
print(len(labeled_val2))

270052
33506
33585


In [ ]:
labeled_train2.iloc[0, 0]

'he currently works as a fitness trainer .\n'

# Calculate WEAT scores

In [ ]:
pip install wefe

In [ ]:
from wefe import WordEmbeddingModel, Query, WEAT
from wefe.utils import run_queries
from gensim.models import Word2Vec

In [ ]:
concat_df1 = pd.concat([labeled_train1, labeled_test1, labeled_val1])
concat_df2 = pd.concat([labeled_train2, labeled_test2, labeled_val2])

#sampled_df1 = concat_df1.groupby('gender').apply(lambda x: x.sample(n=7000)).reset_index(drop = True)

text1 = '\n'.join(concat_df1['text'])
sentences1 = text1.split('\n')
sentences1 = [x.split() for x in sentences1]

text2 = '\n'.join(concat_df2['text'])
sentences2 = text2.split('\n')
sentences2 = [x.split() for x in sentences2]

my_model = Word2Vec(sentences1)
word_vectors = my_model.wv
word2vec_model = WordEmbeddingModel(word_vectors)

In [ ]:
gender_good_bad_query = Query([female_lst, male_lst],
                                [good_lst, bad_lst],
                                ['Females', 'Males'],
                                ['Good', 'Bad'])

weat = WEAT()
result = weat.run_query(gender_good_bad_query, word2vec_model, lost_vocabulary_threshold = 0.4)

print(result)

{'query_name': 'Females and Males wrt Good and Bad', 'result': 0.21606764127285433, 'weat': 0.21606764127285433, 'effect_size': 0.20774979374388233, 'p_value': nan}


In [ ]:
my_model2 = Word2Vec(sentences2)
word_vectors2 = my_model2.wv
word2vec_model2 = WordEmbeddingModel(word_vectors2)

In [ ]:
# Create a query for comparing different pairs of attribute sets
gender_home_work_query = Query(
    [female_lst, male_lst],
    [home_lst, work_lst],
    ['Females', 'Males'],
    ['Home', 'Work'])

gender_art_science_query = Query(
    [female_lst, male_lst],
    [art_lst, science_lst],
    ['Females', 'Males'],
    ['Art', 'Science'])

gender_art_lit_query = Query(
    [female_lst, male_lst],
    [art_lst, literature_lst],
    ['Females', 'Males'],
    ['Art', 'Literature'])

gender_art_math_query = Query(
    [female_lst, male_lst],
    [art_lst, math_lst],
    ['Females', 'Males'],
    ['Art', 'Math'])

gender_science_math_query = Query(
    [female_lst, male_lst],
    [science_lst, math_lst],
    ['Females', 'Males'],
    ['Science', 'Math'])

gender_science_lit_query = Query(
    [female_lst, male_lst],
    [science_lst, literature_lst],
    ['Females', 'Males'],
    ['Science', 'Literature'])

gender_lit_math_query = Query(
    [female_lst, male_lst],
    [literature_lst, math_lst],
    ['Females', 'Males'],
    ['Literature', 'Math'])



queries = [gender_home_work_query, gender_art_science_query, gender_art_lit_query, gender_art_math_query, gender_science_math_query,\
           gender_science_lit_query, gender_lit_math_query]
weat_results = run_queries(WEAT,queries, [word2vec_model2])
weat_results

query_name,Females and Males wrt Home and Work,Females and Males wrt Art and Science,Females and Males wrt Art and Literature,Females and Males wrt Art and Math,Females and Males wrt Science and Math,Females and Males wrt Science and Literature,Females and Males wrt Literature and Math
model_name,,,,,,,
Unnamed model,1.069742,0.931414,0.175856,1.26531,0.333896,-0.755558,1.089454


### Word Embedding

In [ ]:
pip install contractions

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import contractions
import re
import nltk
nltk.download('wordnet')
nltk.download('punkt')
space = ' '

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# replace words in the lists (female, male, good, bad,) to their category words in text
# get ready for word embedding - type 1
def type1_replace_words(text):
    words = text.split()
    for idx in range(len(words)):
        for word_in_list in female_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'female', words[idx])
        for word_in_list in male_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'male', words[idx])
        for word_in_list in good_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'good', words[idx])
        for word_in_list in bad_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'bad', words[idx])
    sentence = space.join(words)
    return sentence

In [ ]:
labeled_train1['replaced'] = labeled_train1['text'].apply(type1_replace_words)
labeled_test1['replaced'] = labeled_test1['text'].apply(type1_replace_words)
labeled_val1['replaced'] = labeled_val1['text'].apply(type1_replace_words)

In [ ]:
# replace words in the sentences with the second group of categories
def type2_replace_words(text):
    words = text.split()
    for idx in range(len(words)):
        for word_in_list in female_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'female', words[idx])
        for word_in_list in male_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'male', words[idx])
        for word_in_list in home_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'home', words[idx])
        for word_in_list in work_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'work', words[idx])
        for word_in_list in art_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'art', words[idx])
        for word_in_list in science_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'science', words[idx])
        for word_in_list in literature_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'literature', words[idx])
        for word_in_list in math_lst:
            words[idx] = re.sub(r"\b%s\b" % word_in_list, 'math', words[idx])
    sentence = space.join(words)
    return sentence

In [ ]:
labeled_train2['replaced'] = labeled_train2['text'].apply(type2_replace_words)
labeled_test2['replaced'] = labeled_test2['text'].apply(type2_replace_words)
labeled_val2['replaced'] = labeled_val2['text'].apply(type2_replace_words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

#### Remove stop words, Puncuation, and perform lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def remove_sw(text):
    text = text.replace(r'[^A-Za-z0-9 \']+', '')
    text = text.replace(r'  ', ' ')
    filtered = []
    word_tokens = word_tokenize(text)
    for w in word_tokens:
        if w not in stop_words:
            # perform lemmatization
            final_w = lemmatizer.lemmatize(w)
            filtered.append(final_w)
    filtered_join = space.join(filtered)
    return filtered_join

In [ ]:
labeled_train1.head(5)

,text,gender,category,replaced
4,she was a member of the district committee of ...,Female,Bad,female was a member of the district committee ...
23,"on 5 november 1994 , he made english football ...",Male,Bad,"on 5 november 1994 , male made english footbal..."
57,he was a national selector at the time of his ...,Male,Bad,male was a national selector at the time of ma...
76,he believed his reputation as a hired killer w...,Male,Bad,male believed male reputation as a hired bad w...
116,"he advocates immigration restrictions , the de...",Male,Bad,"male advocates immigration restrictions , the ..."


In [ ]:
## type 1
labeled_train1['replaced'] = labeled_train1['replaced'].apply(remove_sw)
labeled_test1['replaced'] = labeled_test1['replaced'].apply(remove_sw)
labeled_val1['replaced'] = labeled_val1['replaced'].apply(remove_sw)

## type 2
labeled_train2['replaced'] = labeled_train2['replaced'].apply(remove_sw)
labeled_test2['replaced'] = labeled_test2['replaced'].apply(remove_sw)
labeled_val2['replaced'] = labeled_val2['replaced'].apply(remove_sw)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
labeled_train1.head(5)

,text,gender,category,replaced
4,she was a member of the district committee of ...,Female,Bad,female member district committee communist par...
23,"on 5 november 1994 , he made english football ...",Male,Bad,"5 november 1994 , male made english football h..."
57,he was a national selector at the time of his ...,Male,Bad,male national selector time male bad .
76,he believed his reputation as a hired killer w...,Male,Bad,male believed male reputation hired bad would ...
116,"he advocates immigration restrictions , the de...",Male,Bad,"male advocate immigration restriction , bad pe..."


In [ ]:
labeled_train2.iloc[0, 0]

'he currently works as a fitness trainer .\n'

In [ ]:
labeled_train2.iloc[0, 3]

'male currently work fitness trainer .'

In [ ]:
replaced_all1 = list(labeled_train1['replaced']) + list(labeled_test1['replaced']) + list(labeled_val1['replaced'])
replaced_all2 = list(labeled_train2['replaced']) + list(labeled_test2['replaced']) + list(labeled_val2['replaced'])

In [ ]:
import gensim.models
from gensim.test.utils import datapath
from gensim import utils

In [ ]:
class MyCorpus1:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for line in replaced_all1:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)
class MyCorpus2:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for line in replaced_all2:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

#### Similarity Scores

In [ ]:
import gensim.models

sentences1 = MyCorpus1()
model_type1 = gensim.models.Word2Vec(sentences=sentences1)

print(sentences1)

#type 1
t1_pairs = [
    ('male', 'good'),
    ('male', 'bad'),
    ('female', 'good'),
    ('female', 'bad')
]
for w1, w2 in t1_pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model_type1.wv.similarity(w1, w2)))


'male'	'good'	0.20
'male'	'bad'	0.14
'female'	'good'	0.09
'female'	'bad'	0.11


In [ ]:
sentences2 = MyCorpus2()
model_type2 = gensim.models.Word2Vec(sentences=sentences2)

print(sentences2)
#type 2, home, work, art, science, reading, math
t2_pairs = [
    ('male', 'home'),
    ('male', 'work'),
    ('male', 'art'),
    ('male', 'science'),
    ('male', 'literature'),
    ('male', 'math'),
    ('female', 'home'),
    ('female', 'work'),
    ('female', 'art'),
    ('female', 'science'),
    ('female', 'literature'),
    ('female', 'math')
]
for w1, w2 in t2_pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model_type2.wv.similarity(w1, w2)))

'male'	'home'	0.23
'male'	'work'	0.33
'male'	'art'	0.26
'male'	'science'	0.25
'male'	'literature'	0.30
'male'	'math'	0.26
'female'	'home'	0.16
'female'	'work'	0.16
'female'	'art'	0.36
'female'	'science'	0.08
'female'	'literature'	0.27
'female'	'math'	0.14
